In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils_gen as u

plt.rcParams['figure.figsize'] = (12, 9)
pd.set_option('precision', 2)

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
conn = u.load_connection()

In [4]:
sql_email_status = '''
SELECT *
FROM cio.customer_event
WHERE timestamp >= '2018-11-04'
AND timestamp < '2019-02-24'
'''

df_subscription_status = pd.read_sql(sql_email_status, conn)

In [5]:
df_subscribers = df_subscription_status[df_subscription_status['event_type'] == 'customer_subscribed']
df_unsubscribers = df_subscription_status[df_subscription_status['event_type'] == 'customer_unsubscribed']

In [6]:
all_users_byweek = df_subscription_status.set_index('timestamp').groupby(pd.Grouper(freq='W')).agg(
    {'customer_id': lambda x: len(x.unique())})

In [7]:
unsubscribed_users_byweek = df_unsubscribers.set_index('timestamp').groupby(pd.Grouper(freq='W')).agg(
    {'customer_id': lambda x: len(x.unique())})

In [8]:
(unsubscribed_users_byweek / all_users_byweek) * 100

,customer_id
timestamp,
2018-11-04,46.95
2018-11-11,46.39
2018-11-18,69.68
2018-11-25,58.30
2018-12-02,64.98
2018-12-09,69.61
2018-12-16,40.75
2018-12-23,49.88
2018-12-30,57.73


In [9]:
sql_email_event  = '''
SELECT *
FROM cio.email_sent
WHERE timestamp < '{}'
AND timestamp >= '{}'
AND customer_id in ({})
'''

In [10]:
final_df = pd.DataFrame()

In [11]:
for group, df in df_unsubscribers.set_index('timestamp').groupby(pd.Grouper(freq='W')):
    unsubscribers = u.sql_stringify_list(df['customer_id'].unique())
    
    df_emails_sub = pd.read_sql(sql_email_event.format(group, group - pd.Timedelta(days=7), unsubscribers), conn)
    
    num_emails_peruser = df_emails_sub.groupby('customer_id')['event_id'].size().reset_index()
    num_unsubscribers = len(df['customer_id'].unique())
    
    zero_emails = num_unsubscribers - num_emails_peruser['event_id'].value_counts().sum()
    
    more_than1 = 0
    row_dict = {
        '0': zero_emails / num_unsubscribers,
    }
    for i in range(1, 6):
        more_than1 += num_emails_peruser['event_id'].value_counts()[i]
        row_dict[str(i)] = (num_emails_peruser['event_id'].value_counts()[i] / num_unsubscribers) * 100
    
    row_dict['6+'] = ((num_unsubscribers - zero_emails - more_than1) / num_unsubscribers) * 100
    
    print('num unsubscribed:', len(df['customer_id'].unique()))
    print(group - pd.Timedelta(days=7), group)
    print(num_emails_peruser['event_id'].value_counts())
    print('no emails:', zero_emails)
    print('\n')
    
    final_df = final_df.append(pd.Series(row_dict, name=group))

num unsubscribed: 223
2018-10-28 00:00:00 2018-11-04 00:00:00
3    72
1    54
4    35
2    22
5     5
6     2
Name: event_id, dtype: int64
no emails: 33


num unsubscribed: 7950
2018-11-04 00:00:00 2018-11-11 00:00:00
1     2171
3     2123
2     1948
4     1140
5       83
6       26
7       10
8        5
10       4
9        3
11       2
Name: event_id, dtype: int64
no emails: 435


num unsubscribed: 5866
2018-11-11 00:00:00 2018-11-18 00:00:00
2     2113
1     1707
3     1252
4      230
5       67
6       31
7       29
8       16
9        6
10       2
Name: event_id, dtype: int64
no emails: 413


num unsubscribed: 3259
2018-11-18 00:00:00 2018-11-25 00:00:00
1    1822
2     955
3      63
5      37
6      30
4      28
7      13
9       3
8       3
Name: event_id, dtype: int64
no emails: 305


num unsubscribed: 5650
2018-11-25 00:00:00 2018-12-02 00:00:00
2     2417
1     1923
3      702
5       86
4       79
6       41
7       24
8        9
9        6
10       4
Name: event_id, dtype: i

In [12]:
final_df

,0,1,2,3,4,5,6+
2018-11-04,0.15,24.22,9.87,32.29,15.70,2.24,0.90
2018-11-11,0.05,27.31,24.50,26.70,14.34,1.04,0.63
2018-11-18,0.07,29.10,36.02,21.34,3.92,1.14,1.43
2018-11-25,0.09,55.91,29.30,1.93,0.86,1.14,1.50
2018-12-02,0.06,34.04,42.78,12.42,1.40,1.52,1.49
2018-12-09,0.06,43.76,33.23,15.51,0.48,0.40,0.37
2018-12-16,0.17,48.81,27.30,6.35,0.51,0.09,0.05
2018-12-23,0.07,36.10,30.70,17.73,6.48,1.19,0.69
2018-12-30,0.09,33.47,50.18,7.32,0.21,0.08,0.03
2019-01-06,0.19,47.89,25.54,7.37,0.36,0.04,0.04
